<a href="https://colab.research.google.com/github/Ronbragaglia/An-lise-Automatizada-de-Documentos-com-Azure-AI-e-Machine-Learning/blob/main/An%C3%A1lise_Automatizada_de_Documentos_com_Azure_AI_e_Machine_Learning_para_Detec%C3%A7%C3%A3o_de_Fraudes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install azure-ai-formrecognizer pyzbar opencv-python-headless pandas scikit-learn joblib


import os

os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"] = "https://SEU_ENDPOINT.cognitiveservices.azure.com/"
os.environ["AZURE_FORM_RECOGNIZER_KEY"] = "SUA_CHAVE"

from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

endpoint = os.environ["AZURE_FORM_RECOGNIZER_ENDPOINT"]
key = os.environ["AZURE_FORM_RECOGNIZER_KEY"]

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)


from google.colab import files

uploaded = files.upload()
file_path = list(uploaded.keys())[0]
print(f"📄 Documento carregado: {file_path}")

with open(file_path, "rb") as fd:
    poller = document_analysis_client.begin_analyze_document("prebuilt-document", document=fd)
result = poller.result()

texto_documento = " ".join([line.content for page in result.pages for line in page.lines])
print("🔹 Texto extraído:")
print(texto_documento[:500], "...")


import cv2
from pyzbar.pyzbar import decode

def verificar_autenticidade(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return False
    qrcodes = decode(img)
    return bool(qrcodes)

autenticidade = verificar_autenticidade(file_path)
print("✅ QR Code detectado!" if autenticidade else "⚠️ Sem QR Code detectado.")


import pandas as pd


palavras_suspeitas = ["falso", "teste", "fake", "invalidado"]
tem_palavra_suspeita = int(any(p in texto_documento.lower() for p in palavras_suspeitas))

features_doc = pd.DataFrame([{
    "tamanho_texto": len(texto_documento),
    "tem_qrcode": int(autenticidade),
    "tem_palavra_suspeita": tem_palavra_suspeita
}])

print("🔹 Features extraídas:")
print(features_doc)


from sklearn.ensemble import RandomForestClassifier
import joblib
import numpy as np

X_train = np.array([
    [500, 1, 0],
    [300, 0, 1],
    [800, 1, 0],
    [200, 0, 1],
    [1000, 1, 0]
])
y_train = np.array([0, 1, 0, 1, 0])

modelo_fraude = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_fraude.fit(X_train, y_train)

joblib.dump(modelo_fraude, "modelo_fraude.joblib")


modelo_fraude = joblib.load("modelo_fraude.joblib")
score_fraude = modelo_fraude.predict_proba(features_doc)[0][1]
risco_fraude = "Suspeito de Fraude" if score_fraude > 0.5 else "Autêntico"

print(f"\n🔹 Score de Fraude: {score_fraude:.2f}")
print(f"🔹 Classificação Final: {risco_fraude}")


df_log = pd.DataFrame([{
    "Documento": file_path,
    "Autenticidade_QR": autenticidade,
    "Score_Fraude": round(score_fraude, 2),
    "Classificacao_Final": risco_fraude
}])

df_log.to_csv("log_resultado_avancado.csv", index=False)
print("📄 Log salvo como log_resultado_avancado.csv")
